In [3]:
pip install tsfel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 59.6 MB/s eta 0:00:00


In [4]:
from zipfile import ZipFile
import pandas as pd
import numpy as np
from tensorflow.keras.utils import to_categorical
import tsfel

In [5]:
zip_ref = ZipFile('/content/drive/MyDrive/Colab Notebooks/Deep Learning /Human Activity Recognition/human+activity+recognition+using+smartphones.zip', 'r')
zip_ref.extractall()
zip_ref.close()

In [6]:
zip_ref = ZipFile('/content/UCI HAR Dataset.zip', 'r')
zip_ref.extractall()
zip_ref.close()

In [7]:
def load_raw_data(split):
    base_path = 'UCI HAR Dataset'
    components = ['body_acc', 'body_gyro', 'total_acc']
    axes = ['x', 'y', 'z']
    data = []
    for comp in components:
        for ax in axes:
            file_path = f'/content/{base_path}/{split}/Inertial Signals/{comp}_{ax}_{split}.txt'
            data_ax = np.loadtxt(file_path)
            data.append(data_ax)
    # Stack along the third axis (n_samples, 128, 9)
    return np.stack(data, axis=-1)

In [8]:
X_train_raw = load_raw_data('train')
X_test_raw = load_raw_data('test')

In [9]:
print(X_train_raw.shape)
print(X_test_raw.shape)

(7352, 128, 9)
(2947, 128, 9)


In [10]:
y_train = np.loadtxt('/content/UCI HAR Dataset/train/y_train.txt') - 1
y_test = np.loadtxt('/content/UCI HAR Dataset/test/y_test.txt') - 1
y_train_dl = to_categorical(y_train)
y_test_dl = to_categorical(y_test)

In [11]:
y_train

array([4., 4., 4., ..., 1., 1., 1.])

In [12]:
# Convert to integer type
y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [13]:
y_train_dl = y_train_dl.astype(int)
y_test_dl = y_test_dl.astype(int)

In [14]:
y_test_dl

array([[0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0],
       ...,
       [0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0]])

In [15]:
y_train

array([4, 4, 4, ..., 1, 1, 1])

In [16]:
# 0 - WALKING
# 1 - WALKING_UPSTAIRS
# 2 - WALKING_DOWNSTAIRS
# 3 - SITTING
# 4 - STANDING
# 5 - LAYING

In [17]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from keras.regularizers import L1L2

In [ ]:
def _count_classes(y):
    return len(set(y))

In [ ]:
timesteps = len(X_train_raw[0])
input_dim = len(X_train_raw[0][0])
n_classes = _count_classes(y_train)

print(timesteps)
print(input_dim)
print(len(X_train_raw))
print(n_classes)

128
9
7352
6


In [ ]:
# Initializing parameters
n_epochs = 15
n_batch = 16
n_classes = _count_classes(y_train)

# Bias regularizer value - we will use elasticnet
reg = L1L2(0.01, 0.01)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(X_train_raw, y_train_dl, test_size=0.2, random_state=42)

In [ ]:
X_train.shape

(5881, 128, 9)

In [ ]:
X_val.shape

(1471, 128, 9)

In [ ]:
# Model execution
model = Sequential()
model.add(LSTM(64, input_shape=(timesteps, input_dim), return_sequences=True, bias_regularizer=reg))
model.add(BatchNormalization())
model.add(Dropout(0.50))
model.add(LSTM(48))
model.add(Dropout(0.50))
model.add(Dense(n_classes, activation='sigmoid'))
print("Model Summary: ")
model.summary()

Model Summary: 


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                        │ (None, 128, 64)             │          18,944 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 128, 64)             │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 128, 64)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, 48)                  │          21,696 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 48)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 6)                   │             294 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 41,190 (160.90 KB)

 Trainable params: 41,062 (160.40 KB)

 Non-trainable params: 128 (512.00 B)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_train, Y_train, batch_size=n_batch, validation_data=(X_val, Y_val), epochs=n_epochs)

Epoch 1/20
368/368 ━━━━━━━━━━━━━━━━━━━━ 52s 126ms/step - accuracy: 0.5953 - loss: 2.1846 - val_accuracy: 0.7335 - val_loss: 1.4242
Epoch 2/20
368/368 ━━━━━━━━━━━━━━━━━━━━ 79s 119ms/step - accuracy: 0.8910 - loss: 0.9726 - val_accuracy: 0.9334 - val_loss: 0.5674
Epoch 3/20
368/368 ━━━━━━━━━━━━━━━━━━━━ 85s 129ms/step - accuracy: 0.9276 - loss: 0.5503 - val_accuracy: 0.9007 - val_loss: 0.3998
Epoch 4/20
368/368 ━━━━━━━━━━━━━━━━━━━━ 82s 128ms/step - accuracy: 0.9244 - loss: 0.3226 - val_accuracy: 0.9259 - val_loss: 0.1811
Epoch 5/20
368/368 ━━━━━━━━━━━━━━━━━━━━ 82s 128ms/step - accuracy: 0.9371 - loss: 0.1943 - val_accuracy: 0.7220 - val_loss: 0.9073
Epoch 6/20
368/368 ━━━━━━━━━━━━━━━━━━━━ 80s 124ms/step - accuracy: 0.8536 - loss: 0.4038 - val_accuracy: 0.9572 - val_loss: 0.1311
Epoch 7/20
368/368 ━━━━━━━━━━━━━━━━━━━━ 85s 133ms/step - accuracy: 0.9416 - loss: 0.1648 - val_accuracy: 0.9381 - val_loss: 0.1308
Epoch 8/20
368/368 ━━━━━━━━━━━━━━━━━━━━ 46s 126ms/step - accuracy: 0.9204 - loss: 0

In [ ]:
score = model.evaluate(X_test_raw, y_test_dl)
print(score)

93/93 ━━━━━━━━━━━━━━━━━━━━ 3s 36ms/step - accuracy: 0.8949 - loss: 0.2820
[0.20048795640468597, 0.9304377436637878]


In [18]:
def create_tsfel_config():
    cfg = tsfel.get_features_by_domain()
    return cfg

In [19]:
def extract_sample_features(sample, cfg):
    # Convert sample to DataFrame with sensor columns
    df = pd.DataFrame(sample, columns=[f'sensor_{i}' for i in range(9)])

    # Extract features using parallel processing
    features = tsfel.time_series_features_extractor(cfg,df,fs=100,n_jobs=-1,verbose=0) # n_jobs = -1 uses all cpu cores
    return features.to_numpy().flatten()

In [20]:
from tqdm import tqdm

In [21]:
def extract_features_dataset(X_raw, cfg):
    features = []
    for sample in tqdm(X_raw, desc="Extracting features"):
        features.append(extract_sample_features(sample, cfg))
    return np.array(features)

In [22]:
cfg = create_tsfel_config()

In [23]:
X_train_tsfel = extract_features_dataset(X_train_raw, cfg)
X_test_tsfel = extract_features_dataset(X_test_raw, cfg)

Extracting features: 100%|██████████| 2947/2947 [07:59<00:00,  6.14it/s]


In [24]:
print(X_train_tsfel.shape)
print(X_test_tsfel.shape)

(7352, 1404)
(2947, 1404)


In [25]:
X_train_tsfel

array([[1.76596578e-03, 3.47553975e-03, 2.10000000e+01, ...,
        7.99249912e-04, 4.87628659e-04, 0.00000000e+00],
       [5.06139977e-04, 1.69669290e-03, 2.00000000e+00, ...,
        6.47811902e-04, 3.95649117e-04, 0.00000000e+00],
       [1.10614911e-03, 2.47318321e-03, 2.00000000e+00, ...,
        5.42871337e-04, 3.86669001e-04, 0.00000000e+00],
       ...,
       [8.28560238e+00, 2.47970035e-01, 5.00000000e+00, ...,
        7.96959024e-02, 3.94812820e-02, 6.00000000e+00],
       [8.26689259e+00, 2.46659514e-01, 5.00000000e+00, ...,
        8.26413752e-02, 3.76228103e-02, 8.00000000e+00],
       [7.27404088e+00, 2.19256283e-01, 5.00000000e+00, ...,
        5.83980217e-02, 2.69448984e-02, 5.00000000e+00]])

In [26]:
y_train

array([4, 4, 4, ..., 1, 1, 1])

In [27]:
y_test

array([4, 4, 4, ..., 1, 1, 1])

In [28]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import sklearn.metrics as mt
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier

In [29]:
print(X_test_tsfel.shape)
print(y_test.shape)
print(X_train_tsfel.shape)
print(y_train.shape)

(2947, 1404)
(2947,)
(7352, 1404)
(7352,)


In [30]:
scaler = StandardScaler()
X_train_tsfel = scaler.fit_transform(X_train_tsfel)
X_test_tsfel = scaler.transform(X_test_tsfel)

In [31]:
models = [LogisticRegression(), DecisionTreeClassifier(), RandomForestClassifier(), GradientBoostingClassifier(), KNeighborsClassifier(), XGBClassifier(), SVC()]

In [32]:
best_score = 0
best_model = None
for model in models:
    model.fit(X_train_tsfel, y_train)
    y_pred = model.predict(X_test_tsfel)
    score = model.score(X_test_tsfel, y_test)
    print(f"{model.__class__.__name__}:")
    print("Accuracy:", mt.accuracy_score(y_test, y_pred))
    print("Precision:", mt.precision_score(y_test, y_pred, average='weighted'))
    print("Recall:", mt.recall_score(y_test, y_pred, average='weighted'))
    print("F1-score:", mt.f1_score(y_test, y_pred, average='weighted'))
    print("Score:", score)
    print()
    if score > best_score:
        best_score = score
        best_model = model
    print("Best model updated:", best_model)

LogisticRegression:
Accuracy: 0.9524940617577197
Precision: 0.953316446393949
Recall: 0.9524940617577197
F1-score: 0.9521763667925243
Score: 0.9524940617577197

Best model updated: LogisticRegression()
DecisionTreeClassifier:
Accuracy: 0.8663047166610112
Precision: 0.8655198907452057
Recall: 0.8663047166610112
F1-score: 0.8650083295276423
Score: 0.8663047166610112

Best model updated: LogisticRegression()
RandomForestClassifier:
Accuracy: 0.9311163895486936
Precision: 0.9320000980929078
Recall: 0.9311163895486936
F1-score: 0.9306259697451555
Score: 0.9311163895486936

Best model updated: LogisticRegression()
GradientBoostingClassifier:
Accuracy: 0.9429928741092637
Precision: 0.943385397134996
Recall: 0.9429928741092637
F1-score: 0.9427885432843548
Score: 0.9429928741092637

Best model updated: LogisticRegression()
KNeighborsClassifier:
Accuracy: 0.8914149983033594
Precision: 0.9006865931637169
Recall: 0.8914149983033594
F1-score: 0.8893294941027176
Score: 0.8914149983033594

Best model

In [33]:
print("Best Model:", best_model.__class__.__name__, ", Best Score:", best_score)

Best Model: SVC , Best Score: 0.9531727180183237


# Checking The Metrics for Author's Data

In [34]:
X_train = np.loadtxt('/content/UCI HAR Dataset/train/X_train.txt')
X_test = np.loadtxt('/content/UCI HAR Dataset/test/X_test.txt')

In [35]:
X_train.shape

(7352, 561)

In [36]:
X_test.shape

(2947, 561)

In [37]:
best_score = 0
best_model = None
for model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    score = model.score(X_test, y_test)
    print(f"{model.__class__.__name__}:")
    print("Accuracy:", mt.accuracy_score(y_test, y_pred))
    print("Precision:", mt.precision_score(y_test, y_pred, average='weighted'))
    print("Recall:", mt.recall_score(y_test, y_pred, average='weighted'))
    print("F1-score:", mt.f1_score(y_test, y_pred, average='weighted'))
    print("Score:", score)
    print()
    if score > best_score:
        best_score = score
        best_model = model
    print("Best model updated:", best_model)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression:
Accuracy: 0.9586019681031558
Precision: 0.959839185281027
Recall: 0.9586019681031558
F1-score: 0.9584686641988832
Score: 0.9586019681031558

Best model updated: LogisticRegression()
DecisionTreeClassifier:
Accuracy: 0.8673227010519172
Precision: 0.8689224740541259
Recall: 0.8673227010519172
F1-score: 0.8666527389653171
Score: 0.8673227010519172

Best model updated: LogisticRegression()
RandomForestClassifier:
Accuracy: 0.9331523583305056
Precision: 0.9342852490558377
Recall: 0.9331523583305056
F1-score: 0.9329587625777302
Score: 0.9331523583305056

Best model updated: LogisticRegression()
GradientBoostingClassifier:
Accuracy: 0.9379029521547336
Precision: 0.9387369814279826
Recall: 0.9379029521547336
F1-score: 0.9377400729077036
Score: 0.9379029521547336

Best model updated: LogisticRegression()
KNeighborsClassifier:
Accuracy: 0.9015948422124194
Precision: 0.905930491749607
Recall: 0.9015948422124194
F1-score: 0.9006983259628749
Score: 0.9015948422124194

Best model

In [38]:
print("Best Model:", best_model.__class__.__name__, ", Best Score:", best_score)

Best Model: LogisticRegression , Best Score: 0.9586019681031558
